In [1]:
import sys
sys.path.insert(0, '..')
import numpy as np
import torch
import trimesh
import utils
import math

In [2]:
def generate_sphere_mesh(length):

    assert length >= 2
    width = height = length 
    height = int(height)
    width = int(width)
    X, Y = np.meshgrid(range(width), range(height))
    h = np.stack([X.reshape(-1), Y.reshape(-1)])
    vertices = h.T
    vertices = vertices / (length - 1)
    assert vertices.max() == 1 and vertices.min() == 0

    faces = []

    for y in range(height - 1): 
        if y == height - 2:
            y = y
        for x in range(width - 1):
            if x == width - 2:
                x = 0
            faces.append([x + width * y,  x + 1 + width * y, x + 1 + width * (y + 1)])
            faces.append([x + width * y,  x + 1 + width * (y + 1), x + width * (y + 1)])
    faces = np.asarray(faces)

    return vertices, faces

In [3]:
angles = utils.sample_spherical_angles(dim=3, sample_num=10, spherical_coord=False)[0].numpy()
print(angles.shape)
#rint(coord)

mesh = trimesh.creation.uv_sphere(theta=np.linspace(0., np.pi, 10), phi=np.linspace(-np.pi, np.pi, 10)[:-1])
print(mesh.vertices.shape, mesh.faces.shape)
#mesh.fill_holes()
print(mesh.is_watertight)
mesh.show()


(100, 2)
(90, 3) (144, 3)
True


In [4]:
print(mesh.vertices.shape, mesh.faces.shape)
mesh = trimesh.creation.icosphere(subdivisions=1)
print(mesh.vertices.shape, mesh.faces.shape)
mesh.show()


(90, 3) (144, 3)
(42, 3) (80, 3)


In [29]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
import sys 
sys.path.insert(0, '..')
import kaolin as kal
from kaolin.datasets import shapenet
from kaolin import rep
from kaolin import conversions
import torch
from torch import nn
import matplotlib.pyplot as plt
import math
from models import periodic_shape_sampler_xyz
from models import super_shape_sampler
from models import super_shape
from models import model_utils
import utils
from losses import custom_chamfer_loss
import numpy as np
import random
import tqdm
from collections import defaultdict
from torch.autograd import Variable
import torch.optim as optim
import pickle
import os
import dotenv
import plotly.graph_objects as go
from visualize import plot
import trimesh
from metrics import metrics_functions
from external.PyTorchEMD import emd

ModuleNotFoundError: No module named 'kaolin'

In [ ]:
dotenv.load_dotenv(verbose=True)

category = 'plane'
cache_root = os.getenv('SHAPENET_KAOLIN_CACHE_ROOT')
shapenet_root = os.getenv('SHAPENET_ROOT')
cache_dir = os.path.join(cache_root, category)

categories = [category]


In [ ]:
point_set = shapenet.ShapeNet_Points(root=shapenet_root, categories=categories, cache_dir=cache_dir, train=True, split=1.)
surface_set = shapenet.ShapeNet_Surface_Meshes(root=shapenet_root, categories=categories, cache_dir=cache_dir, train=True, split=1.)

In [ ]:
m = 4
n = 6
dim = 3
points_sample_num = 3000
sample_idx = 0
train_theta_sample_num = 30

device_type = 'cuda:0'
device = torch.device(device_type)

primitive = super_shape.SuperShapes(m, n, quadrics=True, train_ab=False, dim=dim, transition_range=3)
primitive.to(device)
primitive.eval()
primitive.load_state_dict(torch.load('primitive_self_overlap_reg.pth', map_location=device))

sampler = periodic_shape_sampler_xyz.PeriodicShapeSamplerXYZ(points_sample_num, m, n, factor=2, dim=dim)
sampler.to(device)
sampler.eval()
sampler.load_state_dict(torch.load('periodic_sampler_self_overlap_reg.pth', map_location=device))


In [ ]:
icosamesh = trimesh.creation.icosphere(subdivisions=1)
uv = trimesh.util.vector_to_spherical(icosamesh.vertices)
uv[:, 0] = uv[:, 0] * np.pi - np.pi / 2.
uv[:, 1] = uv[:, 1] * 2 * np.pi - np.pi
uv_thetas = torch.tensor(uv).unsqueeze(0).float().to(device)

In [ ]:
points = point_set[sample_idx]['data']['points'].to(device) * 10
all_points_num = points.shape[0]

sampled_points = points[random.sample(range(all_points_num), points_sample_num), :].view(1, -1, dim)
thetas = utils.sample_spherical_angles(batch=1, sample_num=train_theta_sample_num, sampling='grid', device=device, dim=dim, #sgn_convertible=True, phi_margin=1e-5, theta_margin=1e-5)
sgn_convertible=True, phi_margin=0, theta_margin=0)
pred_points, pred_mask, pred_tsd = sampler(primitive(), points=sampled_points, thetas=-thetas)

print(thetas[:, :, 0].min(), thetas[:, :, 0].max(), thetas[:, :, 1].min(), thetas[:, :, 1].max())


In [ ]:
mesh_list = []
target_idx = 5
for idx in range(n):
    if not idx == target_idx:
        continue
    mesh = trimesh.creation.uv_sphere(theta=np.linspace(0, np.pi, train_theta_sample_num), phi=np.linspace(-np.pi, np.pi, train_theta_sample_num))

    mesh.vertices = pred_points[0, idx, :, :].detach().cpu().numpy()

    mesh_list.append(mesh)

    if idx == 0 or target_idx:
        meshes = mesh
    else:
        meshes = trimesh.util.concatenate(meshes, mesh)
    #trimesh.repair.fill_holes(mesh)
    print(meshes.is_watertight, meshes.is_convex)
trimesh.repair.fix_winding(meshes)
trimesh.repair.fix_inversion(meshes)
trimesh.repair.fix_normals(meshes)



#kmesh = kal.rep.TriangleMesh.from_tensors(vertices=torch.tensor(meshes.vertices).float(), faces=torch.tensor(meshes.faces).long())
#kmesh.laplacian_smoothing(iterations=10)
#knmesh = trimesh.Trimesh(vertices=kmesh.vertices.cpu().numpy(), faces=kmesh.faces.cpu().numpy())

meshes.export('plane.obj')

meshes.show()


In [ ]:
subdiv = 4
icosamesh = trimesh.creation.icosphere(subdivisions=subdiv)
uv = trimesh.util.vector_to_spherical(icosamesh.vertices)

uv[:, 1] = uv[:, 1] - np.pi / 2
print(uv[:, 0].max(), uv[:, 0].min(), uv[:, 1].max(), uv[:, 1].min())

margin = .1
uv_thetas = torch.tensor(uv).unsqueeze(0).float().to(device)
uv_th = uv_thetas[:, :, 0].clamp(min=(-math.pi + margin), max=(math.pi - margin))
uv_ph = uv_thetas[:, :, 1].clamp(min=(-math.pi / 2 + margin), max=(math.pi / 2 - margin))
#uv_th = torch.where(uv_th.abs() < margin, torch.tensor([margin], device=device), uv_th)
print(uv_th.max(), uv_th.min(), uv_ph.max(), uv_ph.min())
uv_thetas = torch.stack([uv_th, uv_ph], axis=-1)
#uv_thetas = torch.rand_like(uv_thetas)
uv_thetas = torch.where(uv_thetas.abs() < margin, torch.tensor([margin], device=device), uv_thetas) 

In [ ]:
points = point_set[sample_idx]['data']['points'].to(device) * 10
all_points_num = points.shape[0]

sampled_points = points[random.sample(range(all_points_num), points_sample_num), :].view(1, -1, dim)

uv_pred_points, pred_mask, pred_tsd = sampler(primitive(), points=sampled_points, thetas=uv_thetas)



In [ ]:
target_idx = 5
for idx in range(n):
    if not idx == target_idx:
        continue
    mesh2 = trimesh.creation.icosphere(subdivisions=subdiv)
    mesh2.vertices = uv_pred_points[0, idx, :, :].detach().cpu().numpy()

    if idx == 0 or target_idx:
        meshes2 = mesh2
    else:
        meshes2 = trimesh.util.concatenate(meshes2, mesh2)
    #trimesh.repair.fill_holes(mesh)
    print(meshes2.is_watertight, meshes2.is_convex)
    
#trimesh.repair.fix_winding(meshes)
trimesh.repair.fix_inversion(meshes2)
#trimesh.repair.fix_normals(meshes)



#kmesh = kal.rep.TriangleMesh.from_tensors(vertices=torch.tensor(meshes.vertices).float(), faces=torch.tensor(meshes.faces).long())
#kmesh.laplacian_smoothing(iterations=10)
#knmesh = trimesh.Trimesh(vertices=kmesh.vertices.cpu().numpy(), faces=kmesh.faces.cpu().numpy())

meshes2.export('plane2.obj')

meshes2.show()


In [ ]:
#trimesh.smoothing.filter_laplacian(meshes)


In [ ]:
knmesh.show()